In [0]:
! yes | pip3 uninstall opencv-python
! yes | pip3 uninstall opencv-contrib-python
! yes | pip3 install opencv-python==3.4.2.16
! yes | pip3 install opencv-contrib-python==3.4.2.16

Uninstalling opencv-python-3.4.7.28:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/cv2/*
    /usr/local/lib/python3.6/dist-packages/opencv_python-3.4.7.28.dist-info/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libQtCore-9549151f.so.4.8.7
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libQtGui-6d0f14dd.so.4.8.7
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libavcodec-eac15e48.so.58.21.104
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libavformat-b6bcbe33.so.58.17.101
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libavutil-e1b1a17d.so.56.18.102
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libswresample-b4363bfa.so.3.2.100
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libswscale-15b3fdc6.so.5.2.100
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libvpx-81a43c0a.so.5.0.0
Proceed (y/n)?   Successfully uninstalled opencv-python-3.4.7.28
Uninstalling opencv-contrib-python-3.4.3.

In [0]:
# 캐글 API 버전 갱신을 위한 작업... 반드시 버전이 1.5.6 이어야 함
!ls -lha kaggle.json

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

-rw-r--r-- 1 root root 63 Dec  1 04:40 kaggle.json
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
     |████████████████████████████████| 61kB 2.0MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=fcebba8fb9f88c7dcd4acba1c8f65cc05ea70016d412c9782548d3209697a6dc
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle


In [0]:
# 캐글연동을 위한 토큰 입력
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

# 버전이 1.5.6 이 아니면, 진행할 수 없다
! kaggle -v

Kaggle API 1.5.6


In [0]:
! kaggle competitions download -c 2019-ml-finalproject
! unzip 2019-ml-finalproject.zip

 78% 51.0M/65.0M [00:00<00:00, 112MB/s] 
100% 65.0M/65.0M [00:00<00:00, 137MB/s]
Archive:  2019-ml-finalproject.zip
  inflating: Label2Names.csv         
  inflating: sample_submission.csv   
  inflating: testAll_v2/image_0001.jpg  
  inflating: testAll_v2/image_0002.jpg  
  inflating: testAll_v2/image_0003.jpg  
  inflating: testAll_v2/image_0004.jpg  
  inflating: testAll_v2/image_0005.jpg  
  inflating: testAll_v2/image_0006.jpg  
  inflating: testAll_v2/image_0007.jpg  
  inflating: testAll_v2/image_0008.jpg  
  inflating: testAll_v2/image_0009.jpg  
  inflating: testAll_v2/image_0010.jpg  
  inflating: testAll_v2/image_0011.jpg  
  inflating: testAll_v2/image_0012.jpg  
  inflating: testAll_v2/image_0013.jpg  
  inflating: testAll_v2/image_0014.jpg  
  inflating: testAll_v2/image_0015.jpg  
  inflating: testAll_v2/image_0016.jpg  
  inflating: testAll_v2/image_0017.jpg  
  inflating: testAll_v2/image_0018.jpg  
  inflating: testAll_v2/image_0019.jpg  
  inflating: testAll_v2/image

In [0]:
pip install kmc2

     |████████████████████████████████| 102kB 2.3MB/s 
     |████████████████████████████████| 163kB 8.9MB/s 
  Created wheel for kmc2: filename=kmc2-0.1-cp36-cp36m-linux_x86_64.whl size=252230 sha256=d4eb577906e2073f2470088bdaef31de15ecd11bbb07d3906882eb4eb9032cb8
  Stored in directory: /root/.cache/pip/wheels/5c/ba/f0/4c8b421be72d4f2d1a93233c2f6f591e7d8b0bda05a1f4616f
Successfully built kmc2


In [0]:
import os
import cv2
import kmc2
import numpy as np
import pandas as pd
import cupy as xp
from glob import glob
from PIL import Image
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC,SVC
from scipy.cluster.vq import vq
from sklearn.metrics import confusion_matrix
from sklearn import svm

In [0]:
code_book_size = 600
img_size = 256
Step_size = 8
cls_num = 101
des_size = int(img_size/Step_size)
p1 = int(des_size*(1/4))
p2 = int(des_size*(2/4))
p3 = int(des_size*(3/4))

In [0]:
df_data = pd.read_csv('/content/Label2Names.csv', header=None)

In [0]:
# sift 선언
sift = cv2.xfeatures2d.SIFT_create()

In [0]:
DATA_ROOT_TRAIN = "./train"
train_des = list()
train_images = list()
train_labels = list()

for cls in tqdm(os.listdir(DATA_ROOT_TRAIN)):
  img_list = os.listdir(DATA_ROOT_TRAIN+'/'+cls)
  img_list.sort()

  if cls == 'BACKGROUND_Google' :
    label = 102
    if cls_num == 101 :
      continue
  else :
    label = (df_data.index[df_data[1]==cls]+1).tolist()[0]

  for ii,img in enumerate(img_list) :

    image = cv2.imread(DATA_ROOT_TRAIN+'/'+cls+'/'+img)
    gray= cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (img_size,img_size))

    kp = [cv2.KeyPoint(x, y, Step_size) for y in range(0, gray.shape[0], Step_size) for x in range(0, gray.shape[1], Step_size)]
    keypoint, des = sift.compute(gray, kp)

    train_des.append(des)
    train_images.append(gray)
    train_labels.append(label)

train_labels = np.array(train_labels)

In [0]:
np.array(train_des).reshape(-1,128).shape

(3102720, 128)

In [0]:
seeding = kmc2.kmc2(np.array(train_des).reshape(-1,128), code_book_size)
Kmeas = MiniBatchKMeans(code_book_size, init=seeding).fit(np.array(train_des).reshape(-1,128))
codebook = Kmeas.cluster_centers_

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: Explicit initial center position passed: performing only one init in MiniBatchKMeans instead of n_init=3
  
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=600. Setting it to 3*k
  init_size=init_size)


In [0]:
def VLAD(X,visualDictionary):

    predictedLabels = visualDictionary.predict(X)
    centers = visualDictionary.cluster_centers_
    labels=visualDictionary.labels_
    k=visualDictionary.n_clusters
   
    m,d = X.shape
    V=np.zeros([k,d])
    #computing the differences

    # for all the clusters (visual words)
    for i in range(k):
        # if there is at least one descriptor in that cluster
        if np.sum(predictedLabels==i)>0:
            # add the diferences
            V[i]=np.sum(X[predictedLabels==i,:]-centers[i],axis=0)
    

    V = V.flatten()
    # power normalization, also called square-rooting normalization
    V = np.sign(V)*np.sqrt(np.abs(V))

    # L2 normalization

    V = V/np.sqrt(np.dot(V,V))
    return V

In [0]:
VLAD_vector = list()

for i in tqdm(range(len(train_des))) :
  VLAD_vector.append(VLAD(train_des[i],Kmeas))

VLAD_vector = np.array(VLAD_vector)

In [0]:
VLAD_vector.shape

(3030, 76800)

In [0]:
DATA_ROOT_TEST = "./testAll_v2"

test_des = list()
test_images = list()

img_list = os.listdir(DATA_ROOT_TEST)
img_list.sort()

for img in tqdm(img_list):
  image = cv2.imread(DATA_ROOT_TEST+'/'+img)
  gray= cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
  gray = cv2.resize(gray, (img_size,img_size))   

  kp = [cv2.KeyPoint(x, y, Step_size) for y in range(0, gray.shape[0], Step_size) for x in range(0, gray.shape[1], Step_size)]
  keypoint, des = sift.compute(gray, kp)

  test_des.append(des)
  test_images.append(gray)

In [0]:
VLAD_vector_test = list()

for i in tqdm(range(len(test_des))) :
  VLAD_vector_test.append(VLAD(test_des[i],Kmeas))

VLAD_vector_test = np.array(VLAD_vector_test)

In [0]:
train_labels.shape

(3030,)

In [0]:
parameters = {'kernel':('linear'),'C':[0.01, 0.1 , 1, 10, 100]}
svm = svm.SVC(gamma="scale")
clf = GridSearchCV(svm, parameters, cv=5)
clf.fit(VLAD_vector, train_labels)

In [0]:
result = clf.predict(VLAD_vector_test)

In [0]:
result = result.reshape(-1,1)
result_img_list = np.array(img_list).reshape(-1,1)
total_result = np.hstack([result_img_list,result])

In [0]:
df = pd.DataFrame(total_result, columns=["Id","Category"])
df.to_csv('results-jwkim-VLAD.csv',index=False, header=True)

In [0]:
! kaggle competitions submit -c 2019-ml-finalproject -f results-jwkim-VLAD.csv -m "Final_Term_Project"

In [0]:
print(clf.best_score_)
print(clf.best_params_)